# Activity 2.2 - Transfer Learning

#### Objective(s):

This activity aims to introduce how to apply transfer learning

#### Intended Learning Outcomes (ILOs):
* Demonstrate how to build and train neural network
* Demonstrate how to apply transfer learning in neural network


#### Resources:
* Jupyter Notebook
* CIFAR-10 dataset

#### Procedures
Load the necessary libraries

In [54]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Set the parameters

In [55]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

Set how the input data is loaded

In [56]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

* Write a function to include all the training steps.
* Use the model, training set, test set and number of classes as function parameters


In [57]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))

    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

Shuffle and split the data between train and test sets

In [58]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Create two datasets
* one with digits below 5
* one with 5 and above

In [59]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

* Define the feature layers that will used for transfer learning
* Freeze these layers during fine-tuning process

In [60]:
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

Define the classification layers

In [61]:
classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

Create a model by combining the feature layers and classification layers

In [62]:
model = Sequential(feature_layers + classification_layers)

Check the model summary

In [63]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

 Train the  model on the digits 5,6,7,8,9

In [64]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 41s 175ms/step - loss: 1.6073 - accuracy: 0.2310 - val_loss: 1.5941 - val_accuracy: 0.3456
Epoch 2/5
230/230 [==============================] - 39s 169ms/step - loss: 1.5872 - accuracy: 0.2945 - val_loss: 1.5714 - val_accuracy: 0.4394
Epoch 3/5
230/230 [==============================] - 39s 168ms/step - loss: 1.5672 - accuracy: 0.3503 - val_loss: 1.5468 - val_accuracy: 0.5472
Epoch 4/5
230/230 [==============================] - 39s 168ms/step - loss: 1.5440 - accuracy: 0.4072 - val_loss: 1.5194 - val_accuracy: 0.6423
Epoch 5/5
230/230 [==============================] - 39s 168ms/step - loss: 1.5178 - accuracy: 0.4606 - val_loss: 1.4884 - val_accuracy: 0.6984
Training time: 0:03:22.972109
Test score: 1.4884456396102905
Test accuracy: 0.6984159350395203


Freeze only the feature layers

In [65]:
for l in feature_layers:
    l.trainable = False

Check again the summary and observe the parameters from the previous model

In [66]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 activation_4 (Activation)   (None, 26, 26, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 activation_5 (Activation)   (None, 24, 24, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 12, 12, 32)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 32)        0         
                                                      

Train again the model using the 0 to 4 digits

In [67]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 15s 58ms/step - loss: 1.5862 - accuracy: 0.2563 - val_loss: 1.5583 - val_accuracy: 0.3892
Epoch 2/5
240/240 [==============================] - 15s 63ms/step - loss: 1.5525 - accuracy: 0.3397 - val_loss: 1.5234 - val_accuracy: 0.5363
Epoch 3/5
240/240 [==============================] - 14s 58ms/step - loss: 1.5215 - accuracy: 0.4190 - val_loss: 1.4895 - val_accuracy: 0.6708
Epoch 4/5
240/240 [==============================] - 16s 67ms/step - loss: 1.4908 - accuracy: 0.4913 - val_loss: 1.4568 - val_accuracy: 0.7538
Epoch 5/5
240/240 [==============================] - 15s 63ms/step - loss: 1.4601 - accuracy: 0.5569 - val_loss: 1.4240 - val_accuracy: 0.8144
Training time: 0:01:14.749428
Test score: 1.4240431785583496
Test accuracy: 0.8143607974052429


#### Supplementary Activity
Now write code to reverse this training process. That is, you will train on the digits 0-4, and then finetune only the last layers on the digits 5-9.

In [68]:
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Epoch 1/5
240/240 [==============================] - 15s 58ms/step - loss: 1.4304 - accuracy: 0.6139 - val_loss: 1.3922 - val_accuracy: 0.8568
Epoch 2/5
240/240 [==============================] - 14s 57ms/step - loss: 1.3989 - accuracy: 0.6664 - val_loss: 1.3608 - val_accuracy: 0.8854
Epoch 3/5
240/240 [==============================] - 15s 63ms/step - loss: 1.3702 - accuracy: 0.6993 - val_loss: 1.3288 - val_accuracy: 0.8998
Epoch 4/5
240/240 [==============================] - 14s 57ms/step - loss: 1.3396 - accuracy: 0.7364 - val_loss: 1.2967 - val_accuracy: 0.9085
Epoch 5/5
240/240 [==============================] - 14s 58ms/step - loss: 1.3090 - accuracy: 0.7651 - val_loss: 1.2650 - val_accuracy: 0.9171
Training time: 0:01:22.688531
Test score: 1.2650330066680908
Test accuracy: 0.9171044826507568


In [69]:
for l in feature_layers:
    l.trainable = False

In [70]:
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Epoch 1/5
230/230 [==============================] - 15s 64ms/step - loss: 1.4744 - accuracy: 0.4436 - val_loss: 1.4437 - val_accuracy: 0.5904
Epoch 2/5
230/230 [==============================] - 15s 64ms/step - loss: 1.4481 - accuracy: 0.4848 - val_loss: 1.4146 - val_accuracy: 0.6369
Epoch 3/5
230/230 [==============================] - 15s 63ms/step - loss: 1.4202 - accuracy: 0.5283 - val_loss: 1.3861 - val_accuracy: 0.6743
Epoch 4/5
230/230 [==============================] - 15s 63ms/step - loss: 1.3948 - accuracy: 0.5648 - val_loss: 1.3582 - val_accuracy: 0.7007
Epoch 5/5
230/230 [==============================] - 15s 63ms/step - loss: 1.3706 - accuracy: 0.5927 - val_loss: 1.3308 - val_accuracy: 0.7175
Training time: 0:01:13.896661
Test score: 1.3308072090148926
Test accuracy: 0.7175478339195251


#### Conclusion

The model's accuracy increased from 70% to 81% when it was first trained using activity digits that were greater than or equal to 5, frozen, and then trained again using digits less than 5. However, the accuracy of the model dropped drastically from 92% to 72% when we reversed the process, first training it with digits less than 5, then freezing the feature layers, and fine-tuning it with digits greater than or equal to 5. Even though the test score shifts, it is crucial that it declines because this indicates improved performance and indicates how well the model functions with unknown data. Accuracy has been mostly the focus of this exercise, but I believe that other factors, such as training time, may also be significant in real-world applications. Additionally, this activity used the MNIST dataset, which is a very basic dataset consisting only of digits. Through this exercise, it teache us how to use models in the context of transfer learning to enhance performance on a new task.
